This notebook contains all of our ideas for what should be included in the EDA.

In [8]:
#Library Imports --- IGNORE
import numpy as np
import pandas as pd
import matplotlib as plt
import os

from pathlib import Path

In [10]:
#Reading the files
pd.read_excel(os.path.join("..", "Metacritic dataset", "ExpertReviews.xlsx"))
pd.read_excel(os.path.join("..", "Metacritic dataset", "metaClean43Brightspace.xlsx"))
pd.read_excel(os.path.join("..", "Metacritic dataset", "sales.xlsx"))
pd.read_excel(os.path.join("..", "Metacritic dataset", "UserReviews.xlsx"))

,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot
0,https://www.metacritic.com/movie/bronson,8,'Longbottom94',"'Apr 25, 2013'",'Many have dismissed this film for not explor...,2,2
1,https://www.metacritic.com/movie/bronson,9,'MartinB',"'Oct 13, 2009'",'Anyone who doesn t like this movie simply ju...,0,1
2,https://www.metacritic.com/movie/bronson,10,'Jaakko',"'Jul 19, 2012'",'Not sure what to think at this film at first...,1,1
3,https://www.metacritic.com/movie/bronson,6,'CapoR',"'Oct 13, 2009'",'Nicely portrayed but it lacks the elements t...,0,1
4,https://www.metacritic.com/movie/bronson,8,'OrwellB.',"'Oct 10, 2009'",'Bronson is more than entertainment. It is ar...,0,0
...,...,...,...,...,...,...,...
319657,https://www.metacritic.com/movie/spirited-away,10,'Zenflar',"'Sep 6, 2021'",'A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+A+...,0,0
319658,https://www.metacritic.com/movie/spirited-away,10,'juiliopaublito',"'Nov 25, 2021'",'in my opinion spirited away is one of the be...,0,0
319659,https://www.metacritic.com/movie/spirited-away,10,'PorridgeBoy3000',"'Dec 31, 2021'",'Usually I don t like watching movies that mu...,0,0
319660,https://www.metacritic.com/movie/spirited-away,10,'jamesfhall',"'Jan 20, 2022'",'Studio Ghibli s gripping masterpiece Spirite...,0,0
